In [1]:
from UTILITY_quickstart import *

In [2]:
importedDefaultSettings = loadConfig("setLattice_configs/2024-10-22_oneBunch_baseline2.1.yml")

In [3]:
csrTF = True
evalElement = "PENT"
#evalElement = "BEGPDC"


inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"]
bunchCount = importedDefaultSettings["bunchCount"]
tao = initializeTao(
    inputBeamFilePathSuffix = inputBeamFilePathSuffix,
    
    csrTF = csrTF,
    numMacroParticles=1e4,
    scratchPath = "/tmp",
    randomizeFileNames = True
)


#tao.cmd("set ele * space_charge_method = slice")
#tao.cmd("set ele * space_charge_method = fft_3d")



setLattice(tao, **importedDefaultSettings  )

trackBeam(tao, 
          **importedDefaultSettings,
          verbose = True
         )

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-S2E
CSR on
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using /Users/nmajik/Documents/SLAC/FACET2-S2E/setLattice_configs/defaults.yml
Number of macro particles = 10000.0
Beam created, written to /tmp/beams/activeBeamFile_12510777842854075926.h5, and reinit to tao
Loaded /tmp/beams/activeBeamFile_12510777842854075926.h5
Set track_start = L0AFEND, track_end = end
Set track_end = MFFF
Tracking!
Beam centered at MFFF written to /tmp/beams/patchBeamFile_12510777842854075926.h5
Loaded /tmp/beams/patchBeamFile_12510777842854075926.h5
Set track_start = MFFF, track_end = end
Tracking!
trackBeam() exiting


In [4]:
P = getBeamAtElement(tao, "LI19BEG")
makeBeamActiveBeamFile(P, tao = tao) 

In [5]:
setLattice(tao, **importedDefaultSettings  )
trackBeam(tao, 
          trackStart = "LI19BEG",
          trackEnd = "BEGBC20",
          **importedDefaultSettings,
          verbose = True
         )

P = getBeamAtElement(tao, "BEGBC20")

Loaded /tmp/beams/activeBeamFile_12510777842854075926.h5
Set track_start = LI19BEG, track_end = BEGBC20
Tracking!
trackBeam() exiting


In [6]:
# results = []
# for XC19802kG in np.arange(-1e-3, 1e-3, 1e-4): 
#     for XC19900kG in np.arange(-1e-3, 1e-3, 1e-4): 

def objective(params):
    [XC19802kG, XC19900kG] = params
    setLattice(tao, **importedDefaultSettings | {"XC19802kG" : XC19802kG, "XC19900kG" : XC19900kG}  )
    trackBeam(tao, 
              trackStart = "LI19BEG",
              trackEnd = "BEGBC20",
              **importedDefaultSettings
             )
    
    P = getBeamAtElement(tao, "BEGBC20")
    
    specs = getBeamSpecs(P)

    
    print( 1e6 * specs["PDrive_median_x"], 1e6 * specs["PDrive_median_xp"], XC19802kG, XC19900kG )

    return specs["PDrive_median_x"] ** 2 + specs["PDrive_median_xp"] ** 2

In [7]:
objective([0.0, 0.0])

-123.56395128322275 3.796876632130731 0.0 0.0


np.float64(1.5282466328882264e-08)

In [8]:
from scipy.optimize import minimize

result = minimize(
    objective, 
    [0,0], #Starting point
    method='Nelder-Mead'
)

-123.56395315395744 3.7968768406444693 0.0 0.0
-134.2011507140617 3.260526449978391 0.00025 0.0
-127.61680191102711 3.19004219849577 0.0 0.00025
-116.99894858985756 3.72639266035982 -0.00025 0.00025
-108.4067860292732 3.9592096180403638 -0.0005 0.000375
-104.35781848674283 4.566159929657251 -0.0005 0.000125
-92.7191184081562 5.2542186644465865 -0.00075 6.250000000000006e-05
-77.57581117590911 5.416667475234109 -0.00125 0.00043750000000000006
-54.58192516680367 6.226107059228583 -0.001875 0.0006562500000000001
-38.927750187720896 7.521456554047683 -0.0021249999999999997 0.0003437500000000002
-4.158655695757975 9.30252208556625 -0.0029375 0.0003281250000000003
34.054533091433115 10.27483111002446 -0.0040625 0.0009218750000000004
84.57556286668986 13.350824875590256 -0.005125 0.0005937500000000005
-19.80520101866566 8.007628636655456 -0.0026875 0.0006406250000000002
-57.96975089586166 7.035284510300218 -0.0015624999999999997 4.687500000000015e-05
11.021808212300364 9.464970560044211 -0.00

In [9]:
result

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 2.589778369232592e-15
             x: [-5.308e-03  6.278e-03]
           nit: 32
          nfev: 63
 final_simplex: (array([[-5.308e-03,  6.278e-03],
                       [-5.282e-03,  6.208e-03],
                       [-5.277e-03,  6.201e-03]]), array([ 2.590e-15,  5.399e-15,  7.575e-15]))